# Aerial Scene Classification with EfficientNet-B0
Using Custom Dataset and Transfer Learning

## 1. Import Libraries

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns


## 2. Label-Index Mapping

In [ ]:


def label_index(train_csv, test_csv):


    train_data = pd.read_csv(train_csv)
    test_data = pd.read_csv(test_csv)


    train_label = train_data['label'].unique().tolist()

    test_label = test_data['label'].unique().tolist()


    labels = list(set(train_label + test_label))
    labels.sort()  


    li_dir = {label: index for index, label in enumerate(labels)}
    il_dir = {index: label for label, index in li_dir.items()}

    print(f"{len(labels)} categories found: {labels}")
    return li_dir, il_dir


## 3. Define Custom Dataset

In [ ]:


class intial_dataset(Dataset):
    def __init__(self, file_path, li_dir, preprocess=None, root_path=None):
        self.file_data = pd.read_csv(file_path)
        self.li_dir = li_dir
        self.preprocess = preprocess
        self.root_path = root_path

      
        self.image_paths = self.file_data['image_path'].values
        self.label_names = self.file_data['label'].values

    def __len__(self):
        return len(self.file_data)        

    def __getitem__(self, index):
      
        image_path = self.image_paths[index]
        label_name = self.label_names[index]
        label = self.li_dir[label_name]

    
        if self.root_path:
            img_path = os.path.join(self.root_path, image_path)
        else:
            img_path = image_path

        image = Image.open(img_path).convert('RGB')

        if self.preprocess:
            image = self.preprocess(image)

        return image, label


## 4. Define Training Function

In [ ]:


def train_model(model, criterion, optimizer, train_loader, test_loader, device, epochs=5):
  
    train_losses,  test_losses = [], []
    train_acc_list, test_acc_list = [], []

    for epoch in range(epochs):
        model.train()           
        train_loss = 0.0        
        train_correct_num = 0   
        train_total_num = 0

       
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)       
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)      
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)        
            train_correct_num += (predicted == labels).sum().item()
            train_total_num += labels.size(0)

       
        epoch_train_loss = train_loss / train_total_num
        epoch_train_acc = train_correct_num / train_total_num

        
        model.eval()            
        test_loss = 0.0        
        test_correct_num = 0
        test_total_num = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)       
                outputs = model(images)
                loss = criterion(outputs, labels)       

                test_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)       
                test_correct_num += (predicted == labels).sum().item()
                test_total_num += labels.size(0)

        
        epoch_test_loss = test_loss / test_total_num
        epoch_test_acc = test_correct_num / test_total_num

       
        train_losses.append(epoch_train_loss)
        test_losses.append(epoch_test_loss)
        train_acc_list.append(epoch_train_acc)
        test_acc_list.append(epoch_test_acc)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}, "
              f"Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_acc:.4f}")

    return train_losses, test_losses, train_acc_list, test_acc_list


## 5. Load Data and Define Model

In [ ]:

if __name__ == '__main__':

    train_csv = 'COMP9517/augmented_train.csv'
    test_csv = 'COMP9517/test.csv'
    li_dir, il_dir = label_index(train_csv, test_csv)

   
    train_processed = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    test_processed = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

  
    train_dataset = intial_dataset(
        file_path=train_csv,
        li_dir=li_dir,
        preprocess=train_processed,
        root_path='COMP9517'
    )
    test_dataset = intial_dataset(
        file_path=test_csv,
        li_dir=li_dir,
        preprocess=test_processed,
        root_path='COMP9517'
    )

  
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    print(f"Number of training samples: {len(train_dataset)} Number of test samples: {len(test_dataset)}")
    num_classes = len(li_dir)       
    print("Total number of categories:", num_classes)

    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  
    model = models.efficientnet_b0(pretrained=True)

   
    model.classifier[1] = nn.Linear(1280, num_classes)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)       

 
    EPOCHS = 5
    train_losses, test_losses, train_acc, test_acc = train_model(
        model, criterion, optimizer, train_loader, test_loader, device, epochs=EPOCHS
    )

  
   
    plt.figure()
    plt.plot(range(1, EPOCHS+1), train_losses, label='Train Loss')
    plt.plot(range(1, EPOCHS+1), test_losses, label='Test Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training & Testing Loss')
    plt.legend()
    plt.show()


    plt.figure()
    plt.plot(range(1, EPOCHS+1), train_acc, label='Train Acc')
    plt.plot(range(1, EPOCHS+1), test_acc, label='Test Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training & Testing Accuracy')
    plt.legend()
    plt.show()

 
    model.eval()
    predict_labels = []     
    true_labels = []             

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            predict_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

 
    labels_str = [il_dir[x] for x in true_labels]
    preds_str = [il_dir[x] for x in predict_labels]

    
    print("\n************ Classification Report ************")
    print(classification_report(labels_str, preds_str))

 
    matrix = confusion_matrix(labels_str, preds_str)
    plt.figure(figsize=(8, 6))
    sns.heatmap(matrix, annot=True, cmap='Blues', fmt='d',
                xticklabels=sorted(li_dir.keys()),
                yticklabels=sorted(li_dir.keys()))
    plt.title("Confusion Matrix")
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()
